In [1]:
import astroprov
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad
from astroquery.vizier import Vizier

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import time
from astropy.utils.data import clear_download_cache
from astroquery.gaia import Gaia
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from bs4 import BeautifulSoup

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [12]:
clear_download_cache()

In [2]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

Retrieving tables...
Parsing tables...
Done.
external.external.apassdr9
external.external.gaiadr2_geometric_distance
external.external.ravedr5_com
external.external.ravedr5_dr5
external.external.ravedr5_gra
external.external.ravedr5_on
external.external.sdssdr13_photoprimary
external.external.skymapperdr1_master
external.external.tmass_xsc
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
public.public.dual
tap_config.tap_config.coord_sys
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighb

In [3]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
#         print(fileName)
#         print(os.listdir('.'))

In [4]:
def load_wcs_from_file(filename,pixCoord):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [5]:
def queryObjectRegion(outputTable,searchRadius=5*u.arcsecond):
    imageCoords = []
    r = searchRadius
    
#     if inCat == 'dao':
#         xlabel = 'xcentroid'
#         ylabel = 'ycentroid'
#         label = 'id'
        
#     if inCat == 'sex':
#         xlabel = 'X_IMAGE'
#         ylabel = 'Y_IMAGE'
#         label = 'NUMBER'
        
    xlabel = 'RA'
    ylabel = 'DEC'
    label = 'NUMBER'
    
    flux = outputTable['FLUX_BEST']   
    ra = outputTable['RA']
    dec = outputTable['DEC']
    tableLength = len(outputTable)
    xcoord = outputTable['X_IMAGE']
    ycoord = outputTable['Y_IMAGE']
    imageCoords = [[x,y] for x,y in zip(xcoord,ycoord)]
    imageNames = outputTable['DIFF_NAME']
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype','sp')
    customSimbad.remove_votable_fields('coordinates')
    
    values = [['-'],[0],[0],['-'],['-'],[0],
              [0],[0],['-'],[0]]
    print(len(values))
    Names = ['MAIN_ID','RA_d','DEC_d','OTYPE','CLASS','X_IMAGE',
             'Y_IMAGE','flux','IMAGE','index']
    types = ['object','object','float64','object','object','float64',
             'float64','float64','object','int']     
    #result_table = Table([['-'],[0],[0],['-'],[0],[0],[0],['-'],[0]], names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid','flux','IMAGE','index'),dtype=('object','float64','float64','object','float64','float64','float64','object','object'))
    result_table = Table(values,names=Names,dtype=types)

    for x in range(len(imageCoords)):
        c = SkyCoord(ra[x],dec[x],frame='icrs',unit='deg')
        rt = customSimbad.query_region(c,radius=r)
        try:
            c = SkyCoord(ra[x],dec[x],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            #print(rt)
            rowlist = []
            
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            rowlist.append(flux[x])
            rowlist.append(imageNames[x])
            rowlist.append(x)
            result_table.add_row(rowlist)

        except:
            #print('searchfailed')
            #result_table.add_row(['N/A',ra[x],dec[x],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageNames[x],x])
            result_table.add_row(['N/A',ra[x],dec[x],'Object Not Found','0',[imageCoords[x][0]],
                                  [imageCoords[x][1]],flux[x],imageNames[x],x],
                                 mask=[False,False,False,False,True,False,
                                       False,False,False,False])
    
    return result_table

In [6]:
def keywordFunction(tab,keywordList):
    names = tab.colnames
    #print(names)
    goodColNames = {}
    values = []
    for keyword in keywordList:
        goodColName = [x for x in names if x.find(keyword) != -1]
        if len(goodColName) != 0:
            #print(goodColName)
            #ravalues = []
            for x in range(len(tab)):
                values.append(tab[goodColName[0]][x])
            break
    return values

In [24]:
def vizierSearch(tab,cat):
    result_table = []
    indexref = 'none'
    indexes = []
    for x in range(len(tab)):
        index = tab['OBJECT INDEX'][x]
        if index != indexref:
            RA = tab['RA'][x]
            DEC = tab['DEC'][x]
            #print(bigtab['RA_d'][x],bigtab['DEC_d'][x])
            c = SkyCoord(ra=RA*u.deg,dec=DEC*u.deg)
            result_table_list = Vizier.query_region(c,radius=2*u.arcsec,catalog=cat)
            if len(result_table_list) != 0:
                indexes.append(index)
                if len(result_table) == 0:
                    #result_table_list = Vizier.query_region(c,radius=2*u.arcsec,catalog=cat)
                    result_table = result_table_list[0][0]
                else:
                    res_tab = result_table_list[0][0]
                    result_table = vstack([result_table,res_tab])
        indexref = index
        
#     aa = Column([], name='aa')
#     t.add_column(aa, index=0)
    if len(result_table) != 0:
        result_table['index'] = indexes
        return result_table
    else:
        return 'none'

In [8]:
def findSeparation(ra1,dec1,ra2,dec2):
    c1 = SkyCoord(ra=ra1*u.degree,dec=dec1*u.degree)
    c2 = SkyCoord(ra=ra2*u.degree,dec=dec2*u.degree)
    sep = c1.separation(c2)
    separc = sep.arcsec
    return separc

In [9]:
def getTabValues(Tab,index,tabType,origra,origdec):
    rowlist = 'none'
    if tabType == 'simbad':
        otypename = 'OTYPE'
        source = 'SIMBAD'
    if tabType == 'vizier':
        source = 'VIZIER'
        otypename = 'CLASS'
    Tab = Tab[1:]
    df = Tab.to_pandas()
    df2 = df[df[otypename] != 'Object Not Found']
    df3 = df2[df2['index'] == index]
    if len(df3) != 0:
        ra = df3['RA_d'].tolist()
        dec = df3['DEC_d'].tolist()
        otype = df3[otypename].tolist()
        separations = [round(findSeparation(ra1,dec1,origra,origdec),3) for ra1,dec1 in zip(ra,dec)]
        rowlist = [[index,source,0,0,x,y,0,0,'magfilter',sep,z] for x,y,z,sep in zip(ra,dec,otype,separations)]
    return rowlist

In [10]:
def generateRow(Tab,index,origra,origdec,Names,tabType):
    rowlist = 'none'
    columnNames = Tab.colnames
    Tab = Tab[1:]
    df = Tab.to_pandas()
    if tabType == 'simbad':
        df2 = df[df['OTYPE'] != 'Object Not Found']
    else:
        df2 = df
    df3 = df2[df2['index'] == index]
    rows = []
    astroTab = Table.from_pandas(df3)
    #print('tab',index,len(astroTab))
    if len(df3) != 0:
        for x in range(len(astroTab)):
            if tabType != 'simbad':
                ra1 = astroTab['RAJ2000'][x]
                dec1 = astroTab['DEJ2000'][x]
            else:
                ra1 = astroTab['RA_d'][x]
                dec1 = astroTab['DEC_d'][x]
            sep = round(findSeparation(ra1,dec1,origra,origdec),3)
            rowlist = [astroTab[name][x] if name in columnNames else '--' for name in Names]
            num = Names.index('OBJECT INDEX')
            rowlist[num] = index
            num = Names.index('SOURCE')
            rowlist[num] = tabType
            num = Names.index('SEPARATION')
            rowlist[num] = sep # linked to order
            rows.append(rowlist)
        #print('rows',index,len(rows))
    if len(rows) != 0:
        return rows
    else:
        return 'none'

In [11]:
def bigTable(origTab,simbadTab,ps1Tab,wiseTab,lamostTab,commies,epochs):
    origra = origTab['RA'].tolist()
    origdec = origTab['DEC'].tolist()
    origImage = origTab['IMAGE_NAME'].tolist()
    xim = origTab['X_IMAGE']
    yim = origTab['Y_IMAGE']
    origmag = origTab['MAG_BEST']
    magDiff = origTab['magDiff']
    searchTables = [simbadTab,ps1Tab,wiseTab,lamostTab]
    searchTabNames = ['simbad','PS1','AllWiSE','LAMOST']
    #result_table = Table([[0],[0],['-'],['-'],[0],[0],[0],['-'],[0]],names=('RA_d','DEC_d','Vmag','CLASS','xcentroid','ycentroid','flux','IMAGE','index'),dtype=('float64','float64','object','object','float64','float64','float64','object','int'))
    for x in range(len(origra)):
        if x == 0:

            namelist = ['OBJECT INDEX','SOURCE','ObjID','X_IMAGE','Y_IMAGE','RA_d','DEC_d','MAG','MAG_DIFF','No of Epochs','Comments',
                        'OTYPE','CLASS',
                        'magType','SEPARATION','pmRA','pmDE','var','z','Nd','Ns','Epoch','ex','Qual',
                        'W1mag','W2mag', 'W3mag', 'W4mag',
                        'Kmag','gKmag','rKmag', 'iKmag','zKmag', 'yKmag',   
                        'gmag','rmag','imag','zmag','ymag','Jmag', 'Hmag',
                        'snrg', 'snri','snru',   'snrz',          
                        'Target',  'AllWISE']
#             types = ['int','object','float64','float64','float64','float64','float64','float64','object','object','float64',
#                      'object','object','object','object','object','object','object','object',
#                     'object','object','object','object','object','object','object','object']
            types = ['object']*len(namelist)
            rowlistOG = [[x],[origImage[x]],['-'],[xim[x]],[yim[x]],[origra[x]],[origdec[x]],[origmag[x]],[magDiff[x]],[epochs[x]],[comments[x]]]
            difflen = len(namelist)-len(rowlistOG)
            extras = [['--'] for _ in range(difflen)]
            rowlistOG.extend(extras)

            result_table = Table(rowlistOG,names=namelist,dtype=types)
            for tab in range(len(searchTables)):
                rowlist = generateRow(searchTables[tab],x,origra[x],origdec[x],namelist,searchTabNames[tab])
                if rowlist != 'none':
                    for row in rowlist:
                        result_table.add_row(row)
                        print('working')
        else:
            rowlistOG = [x,origImage[x],'-',xim[x],yim[x],origra[x],origdec[x],origmag[x],magDiff[x],epochs[x],comments[x]]
            difflen = len(namelist)-len(rowlistOG)
            extras = ['--' for _ in range(difflen)]
            rowlistOG.extend(extras)
            result_table.add_row(rowlistOG)
            for tab in range(len(searchTables)):
                rowlist = generateRow(searchTables[tab],x,origra[x],origdec[x],namelist,searchTabNames[tab])
                if rowlist != 'none':
                    for row in rowlist:
                        result_table.add_row(row)
    return result_table

In [12]:
def getPanStarCutOut(ra,dec):
    url = 'https://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={}%2C{}&filter=color&filter=g&filter=r&filter=i&filter=z&filter=y&filetypes=stack&auxiliary=data&size={}&output_size=0&verbose=0&autoscale=99.500000&catlist='.format(ra,dec,size)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    th = soup.findAll('td')
    try:
        link = th[1]
        downLink = link.findAll('img')
        downCut = downLink[0]['src']
        loc1 = downCut.find('cgi-bin')
        loc2 = downCut.find('=/') + len('=/')
        if loc1 != -1:
            if loc2 != -1:
                newCut = downCut[:loc1] + downCut[loc2:]

        loc3 = newCut.find('&')
        smallCut = newCut[:loc3]
        http = 'https:'+smallCut
        print(http)
        subprocess.call(['wget','-O','/home/mj1e16/cutOuts/{}'.format(outname),http])
    except:
        print('WARNING - panstarrs image note found ra = {}, dec = {}'.format(ra,dec))


In [13]:
def XraySearch(tab):
    indexref = 'ooo'
    chandra = {}
    rosat = {}
    xmm = {}
    for x in range(len(tab)):
        index = tab['OBJECT INDEX'][x]
        if index != indexref:
            RA = tab['RA'][x]
            DEC = tab['DEC'][x]
            #print(tab['RA_d'][x],tab['DEC_d'][x])
            c = SkyCoord(ra=RA*u.deg,dec=DEC*u.deg)
            result_table1 = Vizier.query_region(c,radius=2*u.arcsec,catalog='Chandra')
            if len(result_table1) > 0:
                chandra[str(x)] = result_table1
                print('Chandra',result_table1,x)
            #print(result_table[0])
            result_table2 = Vizier.query_region(c,radius=10*u.arcsec,catalog='Rosat')
            if len(result_table2) > 0:
                rosat[str(x)] = result_table2
                print('Rosat',result_table2,x)
            #print(result_table[0])
            result_table3 = Vizier.query_region(c,radius=10*u.arcsec,catalog='XMM-Newton')
            if len(result_table3) > 0:
                xmm[str(x)] = result_table3
                print('XMM',result_table3,x)
    #             for y in range(len(result_table3)):
    #                 print(result_table3[y])
        indexref = tab['OBJECT INDEX'][x]
    return([chandra,rosat,xmm])

In [33]:
otherGaiaTab = ascii.read('/home/mj1e16/outTablesImproved/matchplus3Magdiff.csv')

In [19]:
kepler = otherGaiaTab #ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')
indexes = range(len(kepler))
kepler['OBJECT INDEX'] = indexes
xrayobs = XraySearch(kepler)

In [50]:
df = simbadSearch.to_pandas()
df2 = df[df['OTYPE'] != 'Object Not Found']
df2

,MAIN_ID,RA_d,DEC_d,OTYPE,CLASS,X_IMAGE,Y_IMAGE,flux,IMAGE,index
0,-,0,0.0,-,-,0.0,0.0,0.0,-,0.0


In [44]:
LAMOST

'none'

In [34]:
indexes = range(len(otherGaiaTab))
otherGaiaTab['OBJECT INDEX'] = indexes

In [49]:
simbadSearch = queryObjectRegion(otherGaiaTab,searchRadius=2*u.arcsec)

10
INFO: Upgrading Table to masked Table. Use Table.filled() to convert to unmasked table. [astropy.table.table]


In [39]:
ps1 = vizierSearch(otherGaiaTab,'PS1')

In [40]:
wise = vizierSearch(otherGaiaTab,'AllWiSE')

In [41]:
LAMOST = vizierSearch(otherGaiaTab,'LAMOST')

In [42]:
#var = vizierSearch(bigtab,'J/AJ/156/241/')

In [46]:
comments = ['Very Elongated, Likely noise','Probable Transient','Probable Transient','Easily visible counterpart',
            'Easily visible counterpart','Easily visible counterpart','Easily visible counterpart','Easily visible counterpart',
           't shaped, possibly noise','Likely noise','Easily visible counterpart','Easily visible counterpart',
            'Strange shape, likely noise','Strange shape, likely noise','Probable transient','Counterpart visible in median image but not in PanSTARRS',
           'Possible transient, chance of a visible counterpart','Probable transient','Strange shape, likely noise',
           'Saturated counterpart','Probable transient','Easily visible counterpart','Very elongated, probably noise',
           'Easily visible counterpart','Probable transient','probable transient','Easily visible counterpart','Possible transient',
           'Probable transient','Probable transient','Easily visible counterpart','Easily visible counterpart','Easily visible counterpart',
           'Possible transient','Probable transient','Easily visible counterpart']

In [47]:
epochs = [1, 1, 1, 52, 52, 52, 52, 52, 1, 1, 14, 51, 1, 1, 1, 1, 1, 1, 1, 41, 1, 1, 1, 52, 1, 1, 52, 1, 1, 1, 52, 52, 52, 1, 1, 3]

In [52]:
emptyTab = Table.from_pandas(df2)

In [53]:
simbadSearch = emptyTab
LAMOST = emptyTab

In [47]:
comments = ['--' for x in range(len(otherGaiaTab))]
epochs =  [1 for x in range(len(otherGaiaTab))]

In [54]:
newBigTab = bigTable(otherGaiaTab,simbadSearch,ps1,wise,LAMOST,comments,epochs)

In [660]:
ascii.write(newBigTab,'/home/mj1e16/outTablesImproved/newbigtab.csv',format='csv',fast_writer=False)


In [55]:
newBigTab

OBJECT INDEX,SOURCE,ObjID,X_IMAGE,Y_IMAGE,RA_d,DEC_d,MAG,MAG_DIFF,No of Epochs,Comments,OTYPE,CLASS,magType,SEPARATION,pmRA,pmDE,var,z,Nd,Ns,Epoch,ex,Qual,W1mag,W2mag,W3mag,W4mag,Kmag,gKmag,rKmag,iKmag,zKmag,yKmag,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,snrg,snri,snru,snrz,Target,AllWISE
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,kplr2009114174833_ffi-cal.fits[24],-,342.2666,242.125,285.683191587,43.0450402729,16.8003,-3.0803900093307286,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
1,kplr2009114174833_ffi-cal.fits[38],-,683.0833,458.6268,288.391329351,46.0436457164,15.5744,-3.817542693056524,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
2,kplr2009114174833_ffi-cal.fits[58],-,429.9972,24.7407,291.908147604,47.0584971685,15.7586,-4.733674110651155,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
3,kplr2009114204835_ffi-cal.fits[14],-,318.6326,850.0745,281.758279685,46.7126441695,16.3501,-5.658338521792768,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
4,kplr2009114204835_ffi-cal.fits[15],-,695.7756,712.7224,281.60764435,47.4918680304,16.7669,-3.605746252562367,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
5,kplr2009114204835_ffi-cal.fits[2],-,872.8135,244.0135,283.143048883,43.8178082207,16.7253,-5.278683792538309,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
6,kplr2009114204835_ffi-cal.fits[34],-,748.9758,808.0461,285.689076088,48.7216190609,16.9139,-3.1348216901835055,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
7,kplr2009114204835_ffi-cal.fits[55],-,138.0509,863.371,287.913185827,50.4314229628,16.2629,-3.230583337813577,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
8,kplr2009114204835_ffi-cal.fits[5],-,392.062,72.952,284.837718718,42.5143291681,16.7641,-3.748741535732072,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
